<a href="https://colab.research.google.com/github/ohyunju/projects/blob/ohyunju-patch-2/project2/project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Google Colab으로 수행한다면 이 셀을 먼저 실행해주세요
%%capture
import sys

if 'google.colab' in sys.modules:
    # Install packages in Colab
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*
    !pip install eli5
    !pip install lightgbm
    !pip install shap
    !pip install PDPbox

In [1]:
from google.colab import files
upload=files.upload()

Saving winemag-data_first150k.csv to winemag-data_first150k.csv
Saving winemag-data-130k-v2.csv to winemag-data-130k-v2.csv
Saving winequalityN.csv to winequalityN.csv


In [4]:
import pandas as pd

winequalityN= pd.read_csv('winequalityN.csv')
winequalityN

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,white,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,white,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,red,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
6493,red,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,NaN,11.2,6
6494,red,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
6495,red,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [5]:
winequalityN.isnull().sum()

type                     0
fixed acidity           10
volatile acidity         8
citric acid              3
residual sugar           2
chlorides                2
free sulfur dioxide      0
total sulfur dioxide     0
density                  0
pH                       9
sulphates                4
alcohol                  0
quality                  0
dtype: int64

In [6]:
winequalityN.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
type,6497,2,white,4898,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fixed acidity,6487,NaN,NaN,NaN,7.21658,1.29675,3.8,6.4,7,7.7,15.9
volatile acidity,6489,NaN,NaN,NaN,0.339691,0.164649,0.08,0.23,0.29,0.4,1.58
citric acid,6494,NaN,NaN,NaN,0.318722,0.145265,0,0.25,0.31,0.39,1.66
residual sugar,6495,NaN,NaN,NaN,5.44433,4.75812,0.6,1.8,3,8.1,65.8
chlorides,6495,NaN,NaN,NaN,0.0560416,0.035036,0.009,0.038,0.047,0.065,0.611
free sulfur dioxide,6497,NaN,NaN,NaN,30.5253,17.7494,1,17,29,41,289
total sulfur dioxide,6497,NaN,NaN,NaN,115.745,56.5219,6,77,118,156,440
density,6497,NaN,NaN,NaN,0.994697,0.00299867,0.98711,0.99234,0.99489,0.99699,1.03898
pH,6488,NaN,NaN,NaN,3.2184,0.160748,2.72,3.11,3.21,3.32,4.01


In [7]:
corr=winequalityN.corr()
corr_rank=corr.sort_values(by=['quality'],axis=0,ascending=False)
corr_rank['quality']

# alcohol 이 제일 상관관계가 높음                 

quality                 1.000000
alcohol                 0.444319
citric acid             0.085706
free sulfur dioxide     0.055463
sulphates               0.038729
pH                      0.019366
residual sugar         -0.036825
total sulfur dioxide   -0.041385
fixed acidity          -0.077031
chlorides              -0.200886
volatile acidity       -0.265953
density                -0.305858
Name: quality, dtype: float64

In [8]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(winequalityN, test_size=0.2, random_state=2)
print(train.shape, test.shape)

train, val = train_test_split(train, test_size=0.2, random_state=2)
train.shape, val.shape

(5197, 13) (1300, 13)


((4157, 13), (1040, 13))

In [14]:
target = 'quality'
feature = winequalityN.columns.drop([target])
print(feature)
X_train = train[feature]
y_train = train[target]
X_val = val[feature]
y_val = val[target]
X_test = test[feature]
y_test = test[target]

Index(['type', 'fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol'],
      dtype='object')


In [18]:
from sklearn.linear_model import LinearRegression
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

linear = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),  
    LinearRegression()
)
                                     
linear.fit(X_train_encoded, y_train)

# 회귀계수
coefficients = linear.named_steps['linearregression'].coef_
pd.Series(coefficients, feature)

type                    -0.370838
fixed acidity            0.058932
volatile acidity        -1.436759
citric acid              0.017115
residual sugar           0.052722
chlorides               -0.900286
free sulfur dioxide      0.005592
total sulfur dioxide    -0.001394
density                -81.073765
pH                       0.334508
sulphates                0.644006
alcohol                  0.243629
dtype: float64

In [23]:
# 그래디언트 부스팅
from category_encoders import OrdinalEncoder
from sklearn.metrics import r2_score
from xgboost import XGBRegressor

encoder = OrdinalEncoder()
X_train_encoded = encoder.fit_transform(X_train) # 학습데이터
X_val_encoded = encoder.transform(X_val) # 검증데이터
X_test_encoded = encoder.transform(X_test) # 테스트데이터

boosting = XGBRegressor(
    n_estimators=1000,
    objective='reg:squarederror', # default
    learning_rate=0.2,
    n_jobs=-1
)

eval_set = [(X_train_encoded, y_train), 
            (X_val_encoded, y_val)]

boosting.fit(X_train_encoded, y_train, 
          eval_set=eval_set,
          early_stopping_rounds=50
         )

[0]	validation_0-rmse:4.33324	validation_1-rmse:4.36521
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:3.4958	validation_1-rmse:3.52921
[2]	validation_0-rmse:2.83157	validation_1-rmse:2.86783
[3]	validation_0-rmse:2.30729	validation_1-rmse:2.34771
[4]	validation_0-rmse:1.89597	validation_1-rmse:1.93895
[5]	validation_0-rmse:1.57587	validation_1-rmse:1.6219
[6]	validation_0-rmse:1.33061	validation_1-rmse:1.37988
[7]	validation_0-rmse:1.14506	validation_1-rmse:1.19694
[8]	validation_0-rmse:1.008	validation_1-rmse:1.06268
[9]	validation_0-rmse:0.90678	validation_1-rmse:0.962462
[10]	validation_0-rmse:0.833374	validation_1-rmse:0.891223
[11]	validation_0-rmse:0.782527	validation_1-rmse:0.841284
[12]	validation_0-rmse:0.746932	validation_1-rmse:0.805155
[13]	validation_0-rmse:0.722933	validation_1-rmse:0.77937
[14]	validation_0-rmse:0.706626	validation_1-rmse

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.2, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=-1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [24]:
y_pred = boosting.predict(X_test_encoded)
print('R^2', r2_score(y_test, y_pred))

R^2 0.43049834433224465


In [25]:
feature2=winequalityN.columns.drop([target,'density'])
X_train2 = train[feature2]
X_val2 = val[feature2]
X_test2 = test[feature2]

In [26]:
# 그래디언트 부스팅
from category_encoders import OrdinalEncoder
from sklearn.metrics import r2_score
from xgboost import XGBRegressor

encoder2 = OrdinalEncoder()
X_train_encoded2 = encoder2.fit_transform(X_train2) # 학습데이터
X_val_encoded2 = encoder2.transform(X_val2) # 검증데이터
X_test_encoded2 = encoder2.transform(X_test2) # 테스트데이터

boosting = XGBRegressor(
    n_estimators=1000,
    objective='reg:squarederror', # default
    learning_rate=0.2,
    n_jobs=-1
)

eval_set = [(X_train_encoded2, y_train), 
            (X_val_encoded2, y_val)]

boosting.fit(X_train_encoded2, y_train, 
          eval_set=eval_set,
          early_stopping_rounds=50
         )

[0]	validation_0-rmse:4.33324	validation_1-rmse:4.36521
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:3.4958	validation_1-rmse:3.52921
[2]	validation_0-rmse:2.83157	validation_1-rmse:2.86783
[3]	validation_0-rmse:2.30729	validation_1-rmse:2.34771
[4]	validation_0-rmse:1.89597	validation_1-rmse:1.93895
[5]	validation_0-rmse:1.57587	validation_1-rmse:1.6219
[6]	validation_0-rmse:1.33067	validation_1-rmse:1.3803
[7]	validation_0-rmse:1.14516	validation_1-rmse:1.19642
[8]	validation_0-rmse:1.0079	validation_1-rmse:1.0607
[9]	validation_0-rmse:0.90825	validation_1-rmse:0.961785
[10]	validation_0-rmse:0.83418	validation_1-rmse:0.888766
[11]	validation_0-rmse:0.784845	validation_1-rmse:0.840258
[12]	validation_0-rmse:0.749281	validation_1-rmse:0.805021
[13]	validation_0-rmse:0.724299	validation_1-rmse:0.78012
[14]	validation_0-rmse:0.70722	validation_1-rmse:0.

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.2, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=-1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [27]:
y_pred2 = boosting.predict(X_test_encoded2)
print('R^2', r2_score(y_test, y_pred2))

R^2 0.42353466498507264


In [28]:
winequalityN

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,white,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,white,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,red,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
6493,red,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,NaN,11.2,6
6494,red,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
6495,red,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [31]:
condition=(winequalityN['type']=='red')

winequality_red=winequalityN[condition]
winequality_white=winequalityN[~condition]

print(len(winequality_red),len(winequality_white))

1599 4898
